# Mission Corridors

In [1]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import LineString
from datetime import date
import cpi

#pd.set_option('display.max_rows', 1000) 
#pd.set_option('display.max_columns', 1000)
#pd.set_option("display.max_colwidth", None) 

In [12]:


# ========= INPUT / OUTPUT FILES =========
INPUT_GEOJSON = "docs/Mission/mission_recovery.geojson"
OUTPUT_GEOJSON = "docs/Mission/mission_corridor_rectangles.geojson"

# ========= 1. LOAD ORIGINAL POLYGONS =========
gdf = gpd.read_file(INPUT_GEOJSON)

# assume geometry is in EPSG:4326 (lon/lat)
gdf_4326 = gdf.set_crs(4326, allow_override=True)
gdf_m = gdf_4326.to_crs(3857)  # meters for buffering

# ========= 2. DEFINE CORRIDOR LINES (YOUR COORDS) =========
# (lon, lat) order

sixteenth_line_4326 = LineString([
    (-122.42639848568001, 37.76463787050363),   # Dolores & 16th
    (-122.41862447318765, 37.76513520717117)    # Capp & 16th
])

twentyfourth_line_4326 = LineString([
    (-122.42070428208872, 37.752113850440224),  # Valencia & 24th
    (-122.40636144939559, 37.75298767165918)    # Potrero & 24th
])

valencia_line_4326 = LineString([
    (-122.4223937835193, 37.76987633443966),    # Valencia & Duboce
    (-122.42030618333321, 37.74813359059528)    # Valencia & Cesar Chavez
])

corridor_lines_4326 = {
    "16th Street corridor": sixteenth_line_4326,
    "24th Street corridor": twentyfourth_line_4326,
    "Valencia Street corridor": valencia_line_4326,
}

# convert lines to meters CRS
corridor_lines_m = {
    name: gpd.GeoSeries([line], crs=4326).to_crs(3857).iloc[0]
    for name, line in corridor_lines_4326.items()
}

# ========= 3. TINY BUFFER AROUND EACH CORRIDOR =========
# very small buffer so we only grab adjacent polygons
BUFFER_METERS = 25  # tweak if needed

corridor_buffers_m = {
    name: line.buffer(BUFFER_METERS, cap_style=2)  # cap_style=2 => square ends (rectangle-like)
    for name, line in corridor_lines_m.items()
}

# ========= 4. BUILD AGGREGATED RECTANGLES =========
records = []

for name, buf_geom_m in corridor_buffers_m.items():
    # polygons that touch this tiny buffer
    touched = gdf_m[gdf_m.geometry.intersects(buf_geom_m)]

    if touched.empty:
        # skip if nothing intersects
        continue

    # aggregate numeric fields
    sum_2019 = touched["2019Q2"].astype(float).sum()
    sum_2025 = touched["2025Q2"].astype(float).sum()

    if sum_2019 != 0:
        pct_change = (sum_2025 - sum_2019) / sum_2019 * 100.0
    else:
        pct_change = None

    # use the buffer's bounding rectangle for cleaner look
    rect_m = buf_geom_m.envelope
    rect_4326 = gpd.GeoSeries([rect_m], crs=3857).to_crs(4326).iloc[0]

    records.append({
        "area": name,          # keeps UID_FIELD working
        "2019Q2": sum_2019,
        "2025Q2": sum_2025,
        "pct_change": pct_change,
        "geometry": rect_4326,
    })

corridors_gdf = gpd.GeoDataFrame(records, crs=4326)

# ========= 5. SAVE NEW 3-RECTANGLE GEOJSON =========
corridors_gdf.to_file(OUTPUT_GEOJSON, driver="GeoJSON")

print(f"Saved {len(corridors_gdf)} corridor rectangles to {OUTPUT_GEOJSON}")


Saved 3 corridor rectangles to docs/Mission/mission_corridor_rectangles.geojson


In [13]:
import cpi

In [18]:
cpi.inflate(502493,date(2025, 6, 30),to=date(2019, 6, 30))

399025.5005998865